In [1]:
using ForneyLab, LinearAlgebra

The main motivation behind this implementation is to circumvent complications in factor graphs due to nonlinear nodes. As it will be elaborated further, the current solutions to compute messages around delta factors are mostly limited to Gaussian approximations which does not allow inference in many models. This proposal provides a more generic solution.

# Handling the direct nonconjugacies

This section of the implementation has nothing to do with nonlinearities or VMP. Instead, direct nonconjugate relations in factor graphs are handled. Take the example below where the prior on the mean parameter of Gaussian likelihood function is put Gamma distribution. Posterior inference in such a model is carried out with Laplace approximation. We automate Laplace approximation in a gradient ascent setting where the gradients are calculated thanks to autodiff and step sizes are automaticely adjusted according to the goodness of the fit.

In [2]:
g = FactorGraph()

@RV z ~ Gamma(0.5,0.2)
@RV x ~ GaussianMeanVariance(z,1)

placeholder(x, :x)

Variable(:x, Edges:
Edge belonging to variable x: ( gaussianmeanvariance_1.i[out] )----( placeholder_x.i[out] ).
)

In [3]:
ForneyLab.draw(g)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.42.3 (20191010.1750)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 14751997000028184662 -->
 
 14751997000028184662 
 
 clamp_1 
 
<!-- 6538921846429304147 -->
 
 6538921846429304147 
 
 clamp_2 
 
<!-- 17252387475420271337 -->
 
 17252387475420271337 
 
 placeholder_x 
 
<!-- 6069310519203470980 -->
 
 6069310519203470980 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 17252387475420271337--6069310519203470980 -->
 
 17252387475420271337--6069310519203470980 
 
 x 
 1 out 
 1 out 
 
<!-- 10570110008273349783 -->
 
 10570110008273349783 
 
 Gam 
 gamma_1 
 
<!-- 6069310519203470980--10570110008273349783 -->
 
 6069310519203470980--10570110008273349783 
 
 z 
 1 out 
 2 m 
 
<!-- 6908837662408872030 -->
 
 6908837662408872030 
 
 clamp_3 
 
<!-- 6069310519203470980--6908837662408872030 -->
 
 6069310519203470980--6908837662408872030 
 
 clamp_3 
 1 out 
 3 v 
 
<!-- 10570110008273349783--14751997000028184662 -->
 
 10570110008273349783--14751997000028184662 
 
 clamp_1 
 1 out 
 2 a 
 
<!-- 10570110008273349783--6538921846429304147 -->
 
 10570110008273349783--6538921846429304147 
 
 clamp_2 
 1 out 
 3 b

In [4]:
algo = sumProductAlgorithm(z)
source_code = algorithmSourceCode(algo)
println(source_code)

begin

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleSPGammaOutNPP(nothing, Message(Univariate, PointMass, m=0.5), Message(Univariate, PointMass, m=0.2))
messages[2] = ruleSPGaussianMeanVarianceMPNP(Message(Univariate, PointMass, m=data[:x]), nothing, Message(Univariate, PointMass, m=1))

marginals[:z] = messages[1].dist * messages[2].dist

return marginals

end

end # block


In [5]:
# Define algorithm
eval(Meta.parse(source_code));

In [6]:
data = Dict(:x => 2.2)
margianals = step!(data)

Dict{Any,Any} with 1 entry:
  :z => 𝒩(m=1.71, v=1.21)…

## Multivariate extension

The above implementation is not limited to univariate distributions. The solution for multivariate example is given below.

In [7]:
g = FactorGraph()

@RV z ~ Dirichlet([2.0,1.0,3.4])
@RV x ~ GaussianMeanVariance(z,diagm(ones(3)))

placeholder(x, :x, dims=(3,))

Variable(:x, Edges:
Edge belonging to variable x: ( gaussianmeanvariance_1.i[out] )----( placeholder_x.i[out] ).
)

In [8]:
algo = sumProductAlgorithm(z)
source_code = algorithmSourceCode(algo)
println(source_code)

begin

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleSPDirichletOutNP(nothing, Message(Multivariate, PointMass, m=[2.0, 1.0, 3.4]))
messages[2] = ruleSPGaussianMeanVarianceMPNP(Message(Multivariate, PointMass, m=data[:x]), nothing, Message(MatrixVariate, PointMass, m=[1.0 0.0 0.0; 0.0 1.0 0.0; 0.0 0.0 1.0]))

marginals[:z] = messages[1].dist * messages[2].dist

return marginals

end

end # block


In [9]:
# Define algorithm
eval(Meta.parse(source_code));

In [10]:
data = Dict(:x => [3.2,3.2,3.2])
marginals = step!(data)

Dict{Any,Any} with 1 entry:
  :z => 𝒩(m=[3.49, 3.20, 3.83], v=[[0.92, 0.00, 0.00][0.00, 1.00, 0.00][0.00, 0…

In [11]:
var(marginals[:z])

3-element Array{Float64,1}:
 0.9239991520042407
 1.0               
 0.8592106070514604

## Beyond Gaussians

As it is shown above, the posterior is approximated by Gaussian distribution when one of the messages is Gaussian. However, the user may want to use different type of distributions. In this case, we approximate the posterior with a set of samples and corresponding weights instead of known, exponential family distributions. The technique we resort to is celebrated importance sampling. As an importance distribution, the proposed technique employes one of the messages. This refers to an automated process that is drawing samples from a message and determining the weights by assessing the importance of samples on the other message. The weights are then normalized. 

In [12]:
g = FactorGraph()

@RV z ~ Beta(2,5)
@RV x ~ Poisson(z)

placeholder(x, :x)

Variable(:x, Edges:
Edge belonging to variable x: ( poisson_1.i[out] )----( placeholder_x.i[out] ).
)

In [13]:
algo = sumProductAlgorithm(z)
source_code = algorithmSourceCode(algo)
println(source_code)

begin

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleSPBetaOutNPP(nothing, Message(Univariate, PointMass, m=2), Message(Univariate, PointMass, m=5))
messages[2] = ruleSPPoissonLPN(Message(Univariate, PointMass, m=data[:x]), nothing)

marginals[:z] = messages[1].dist * messages[2].dist

return marginals

end

end # block


In [14]:
# Define algorithm
eval(Meta.parse(source_code));

In [15]:
data = Dict(:x => 7)
marginals = step!(data)

Dict{Any,Any} with 1 entry:
  :z => SampleList(s=Number[0.2923980510022137, 0.29603500335192057, 0.24015619…

In [16]:
mean(marginals[:z])

0.6477795490581133

In [17]:
var(marginals[:z])

0.01782426418823527

Importance sampling enables us to compute crucial statistics about this posterior distribution. This constitutes one of the compounds that boost VMP.

# Boosting VMP

Consider a simple model defined as:

$$
p(z) = \mathcal{N}(z;0,1) \\
p(x) = \mathcal{N}(x;0,1) \\
p(y|x,z) = \mathcal{N}(y;x,exp(z))
$$

Assume we want to compute $p(z|y)$. Belief propagation is intractable in this simple model. Even VMP is not straightforward to carry out. Messages around the nonlinear mapping $w = exp(z)$ should be computed. At first glance, the user may consider to use Taylor series approximation of function or unscented transformation. In the forward direction, this would result in a Gaussian message. Thanks to our implementations for nonconjugate inference detailed above, we could approximate $p(w|y)$ by Gaussian distribution. However, this would not resolve several other problems. 1) The message toward $z$ is still undefined and extended KF and unscetented transformations do not suffice to compute it since the message from $w$ to nonlinear node is not symmetric. 2) Free energy calculation requires some statistics that are not analytically computable for Gaussian distributed RV. 3) We want to free the user to choose the function without having specific constraints. So it is quite possible that this mapping may lead to a distribution which is not similar to Gaussian (such as multimodal dist. or discrete dist.).

Our implementation does not make such an assumption and resort to sampling to compute forward messages. The samples drawn from $p(z)$ are first transformed through $exp(.)$ with a set of accompanying equal weights, sum of which is 1. These weights are then importance adjusted to form approximation $q(w)$ to $p(w|y)$. Note that from now on we can compute any expectation required for VMP and free energy calculation by Monte Carlo summation. The backward message to $z$, on the other hand, is sent as a log-pdf function which constitutes an objective together with the prior message for the posterior calculation of $z$. Since the prior is Gaussian, the approximation once again is realized by Laplace approximation.

In [18]:
g = FactorGraph()

@RV z ~ GaussianMeanVariance(0,1)
@RV x ~ GaussianMeanVariance(0,1)

f(z) = 1/exp(z)
@RV w ~ Nonlinear{Sampling}(z,f,n_samples=1000)

@RV y ~ GaussianMeanPrecision(x,w)

placeholder(y, :y)

Variable(:y, Edges:
Edge belonging to variable y: ( gaussianmeanprecision_1.i[out] )----( placeholder_y.i[out] ).
)

In [19]:
ForneyLab.draw()

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.42.3 (20191010.1750)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 12132578434127664834 -->
 
 12132578434127664834 
 
 Nonlinear{Sampling} 
 nonlinear_1 
 
<!-- 18386876702981392331 -->
 
 18386876702981392331 
 
 𝒩 
 gaussianmeanvariance_1 
 
<!-- 12132578434127664834--18386876702981392331 -->
 
 12132578434127664834--18386876702981392331 
 
 z 
 1 out 
 2 in1 
 
<!-- 12280111956203589263 -->
 
 12280111956203589263 
 
 placeholder_y 
 
<!-- 5930877304415489543 -->
 
 5930877304415489543 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 12280111956203589263--5930877304415489543 -->
 
 12280111956203589263--5930877304415489543 
 
 y 
 1 out 
 1 out 
 
<!-- 12542024808637239899 -->
 
 12542024808637239899 
 
 clamp_4 
 
<!-- 7335802173842971150 -->
 
 7335802173842971150 
 
 clamp_2 
 
<!-- 18386876702981392331--7335802173842971150 -->
 
 18386876702981392331--7335802173842971150 
 
 clamp_2 
 1 out 
 3 v 
 
<!-- 17820592952783008199 -->
 
 17820592952783008199 
 
 clamp_1 
 
<!-- 18386876702981392331--17820592952783008199 -->
 
 18386876702981392331--17820592952783008199 
 
 clamp_1 
 1 out 
 2 m 
 
<!-- 5930877304415489543--12132578434127664834 -->
 
 5930877304415489543--12132578434127664834 
 
 w 
 1 out 
 3 w 
 
<!-- 15157718081284142603 -->
 
 15157718081284142603 
 
 𝒩 
 gaussianmeanvariance_2 
 
<!-- 5930877304415489543--15157718081284142603 -->
 
 5930877304415489543--15157718081284142603 
 
 x 
 1 out 
 2 m 
 
<!-- 5956539486982426216 -->
 
 5956539486982426216 
 
 clamp_3 
 
<!-- 15157718081284142603--12542024808637239899 -->
 
 15157718081284142603--12542024808637239899 
 
 clamp_4 
 1 out 
 3 v 
 
<!-- 15157718081284142603--5956539486982426216 -->
 
 15157718081284142603--5956539486982426216 
 
 clamp_3 
 1 out 
 2 m

In [20]:
PosteriorFactorization()

q_x = PosteriorFactor(x, id=:XMF)
q_z = PosteriorFactor(z, id=:ZMF)
algo_mf = variationalAlgorithm(free_energy=true)

source_code = algorithmSourceCode(algo_mf,free_energy=true)
eval(Meta.parse(source_code));

In [21]:
println(source_code)

begin

function stepZMF!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 4))

messages[1] = ruleVBGaussianMeanVarianceOut(nothing, ProbabilityDistribution(Univariate, PointMass, m=0), ProbabilityDistribution(Univariate, PointMass, m=1))
messages[2] = ruleVBGaussianMeanPrecisionW(ProbabilityDistribution(Univariate, PointMass, m=data[:y]), marginals[:x], nothing)
messages[3] = ruleSPNonlinearSInMN(f, messages[2], nothing)
messages[4] = ruleSPNonlinearSOutNG(f, nothing, messages[1], 1000)

marginals[:w] = messages[4].dist * messages[2].dist
marginals[:z] = messages[1].dist * messages[3].dist

return marginals

end

function stepXMF!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBGaussianMeanVarianceOut(nothing, ProbabilityDistribution(Univariate, PointMass, m=0), ProbabilityDistribution(Univariate, PointMass, m=1))
messages[2] = ruleVBGaussianMeanPrecisionM(ProbabilityDistribution(Univariate,

In [22]:
n_its = 5
marginals_mf = Dict()
F_mf = zeros(n_its)

data = Dict(:y => 1.4)

marginals_mf[:z] = ProbabilityDistribution(Univariate, GaussianMeanVariance, m=0, v=1)
marginals_mf[:x] = ProbabilityDistribution(Univariate, GaussianMeanVariance, m=0, v=1)
marginals_mf[:w] = vague(SampleList)

for i = 1:n_its
    stepXMF!(data, marginals_mf)
    stepZMF!(data, marginals_mf)
    F_mf[i] = freeEnergy(data, marginals_mf)
end

In [23]:
marginals_mf

Dict{Any,Any} with 3 entries:
  :w => SampleList(s=[0.39, 1.03, 1.21, 0.95, 2.80, 0.78, 0.59, 0.27, 0.77, 0.8…
  :z => 𝒩(m=-0.07, v=0.70)…
  :x => 𝒩(xi=1.81, w=2.29)…

In [24]:
mean(marginals_mf[:w])

1.2993004960987355

In [25]:
var(marginals_mf[:w])

1.2156671932770537

In [26]:
F_mf

5-element Array{Float64,1}:
 2.030966806026826 
 1.9508509840075696
 1.9532297056574686
 1.9437299542689082
 1.949582575252852 

A more complete example on a hierarchical Gaussian filter variant can be found at https://github.com/biaslab/BoostingVMP/blob/master/HGF%20Filtering.ipynb Free energy may not decrease at every iteration since approximations are based on sampling, a stochastic process. But overall trend is expected to decrease.

## Flexible model specifications beyond Gaussians

This implementation allows users to define customized relations between random variables. These may include conditional statements, loops, etc. They don't have to be a mapping from Gaussian distributed random variables. An example is given below.

In [27]:
g = FactorGraph()

@RV z ~ Dirichlet([2.1, 4.4, 3.2])
@RV x ~ Categorical(z)

function f(x)
    if x[1] == 1
        w = 0.1
    elseif x[2] == 1
        w = 1
    elseif x[3] == 1
        w = 10
    end
    return w
end

@RV w ~ Nonlinear{Sampling}(x,f,n_samples=1000)

@RV y ~ GaussianMeanPrecision(0,w)

placeholder(y, :y)

Variable(:y, Edges:
Edge belonging to variable y: ( gaussianmeanprecision_1.i[out] )----( placeholder_y.i[out] ).
)

In [28]:
ForneyLab.draw()

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.42.3 (20191010.1750)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 1403768041815337489 -->
 
 1403768041815337489 
 
 𝒩 
 gaussianmeanprecision_1 
 
<!-- 9995352409888602420 -->
 
 9995352409888602420 
 
 clamp_2 
 
<!-- 1403768041815337489--9995352409888602420 -->
 
 1403768041815337489--9995352409888602420 
 
 clamp_2 
 1 out 
 2 m 
 
<!-- 12083987196495509931 -->
 
 12083987196495509931 
 
 Nonlinear{Sampling} 
 nonlinear_1 
 
<!-- 1403768041815337489--12083987196495509931 -->
 
 1403768041815337489--12083987196495509931 
 
 w 
 1 out 
 3 w 
 
<!-- 10023894797634638841 -->
 
 10023894797634638841 
 
 Cat 
 categorical_1 
 
<!-- 12083987196495509931--10023894797634638841 -->
 
 12083987196495509931--10023894797634638841 
 
 x 
 1 out 
 2 in1 
 
<!-- 5199444343422996273 -->
 
 5199444343422996273 
 
 placeholder_y 
 
<!-- 5199444343422996273--1403768041815337489 -->
 
 5199444343422996273--1403768041815337489 
 
 y 
 1 out 
 1 out 
 
<!-- 8484893693517670426 -->
 
 8484893693517670426 
 
 Dir 
 dirichlet_1 
 
<!-- 5563013630151947371 -->
 
 5563013630151947371 
 
 clamp_1 
 
<!-- 8484893693517670426--5563013630151947371 -->
 
 8484893693517670426--5563013630151947371 
 
 clamp_1 
 1 out 
 2 a 
 
<!-- 10023894797634638841--8484893693517670426 -->
 
 10023894797634638841--8484893693517670426 
 
 z 
 1 out 
 2 p

In [29]:
PosteriorFactorization()

q_x = PosteriorFactor(x, id=:XMF)
q_z = PosteriorFactor(z, id=:ZMF)
algo_mf = variationalAlgorithm(free_energy=true)

source_code = algorithmSourceCode(algo_mf,free_energy=true)
eval(Meta.parse(source_code));

In [30]:
println(source_code)

begin

function stepZMF!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 2))

messages[1] = ruleVBDirichletOut(nothing, ProbabilityDistribution(Multivariate, PointMass, m=[2.1, 4.4, 3.2]))
messages[2] = ruleVBCategoricalIn1(marginals[:x], nothing)

marginals[:z] = messages[1].dist * messages[2].dist

return marginals

end

function stepXMF!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 4))

messages[1] = ruleVBCategoricalOut(nothing, marginals[:z])
messages[2] = ruleVBGaussianMeanPrecisionW(ProbabilityDistribution(Univariate, PointMass, m=data[:y]), ProbabilityDistribution(Univariate, PointMass, m=0), nothing)
messages[3] = ruleSPNonlinearSInMN(f, messages[2], nothing)
messages[4] = ruleSPNonlinearSOutNC(f, nothing, messages[1], 1000)

marginals[:w] = messages[4].dist * messages[2].dist
marginals[:x] = messages[1].dist * messages[3].dist

return marginals

end

function freeEnergy(data::Dict, marginals::Dict)

F 

In [34]:
n_its = 5
marginals_mf = Dict()
F_mf = zeros(n_its)

data = Dict(:y => 1.4)

marginals_mf[:z] = ProbabilityDistribution(Dirichlet,a=[1,1,1])
marginals_mf[:x] = ProbabilityDistribution(Categorical,p=[0.3,0.3,0.4])
marginals_mf[:w] = vague(SampleList)

for i = 1:n_its
    stepZMF!(data, marginals_mf)
    stepXMF!(data, marginals_mf)
    F_mf[i] = freeEnergy(data, marginals_mf)
end

In [35]:
marginals_mf

Dict{Any,Any} with 3 entries:
  :w => SampleList(s=`1`, `0.1`, `1`, `0.1`, `1`, `0.1`, `1`, `10`, `1`, `10`, …
  :z => Dir(a=[2.33, 5.17, 3.20])…
  :x => SampleList(s=`  [2]  =  1.0`, `  [3]  =  1.0`, `  [1]  =  1.0`, `  [3] …

In [36]:
F_mf

5-element Array{Float64,1}:
 2.4945508834878956
 2.446900318509765 
 2.4333424039422016
 2.467717087659119 
 2.4331389277036912

During the free energy calculation, the differential entropy of $x$ is required. Note that $q(x)$ does not have a functional form. Differential entropy for this factor is calculated with Monte Carlo summation by using the pdf of incoming messages. Our implementation provides an approximate free energy! Details are given in the paper. Also check a full example with Bernoulli https://github.com/biaslab/BoostingVMP/blob/master/Continuous%20State%20Switching%20Precision.ipynb

## Bivariate function of Gaussian distributed random variables

Lastly, an inference mechanism for bivariate functions of Gaussian distributed random variables is introduced. Similar to what is done above, the forward messages are a set of equally weighted transformed samples. Backward messages are calculated over the joint of input arguments. This is achieved by Laplace approximation once again. Once the joint approximation is achieved, it is easy to compute marginal posteriors since the joint is Gaussian distribution. Later on we compute the backward Gaussian messages towards input arguments by using the incoming Gaussian messages and Gaussian posterior marginal approximations. A simple example is given below. We refer the reader to https://github.com/biaslab/BoostingVMP/blob/master/Bayesian%20parameter%20estimation%20in%20LDS%20Smoothing.ipynb to see how it is used for parameter estimation in a linear dynamical system.

In [37]:
g = FactorGraph()

@RV z ~ GaussianMeanVariance(ones(2),diagm(ones(2)))
@RV x ~ GaussianMeanVariance(ones(2),2*diagm(ones(2)))

f(x,z) = transpose(x)*z
@RV m ~ Bivariate{Sampling}(x,z,f,n_samples=1000)

@RV y ~ GaussianMeanVariance(m,1)

placeholder(y, :y)

Variable(:y, Edges:
Edge belonging to variable y: ( gaussianmeanvariance_3.i[out] )----( placeholder_y.i[out] ).
)

In [38]:
algo = sumProductAlgorithm([x,z,m])
source_code = algorithmSourceCode(algo)
println(source_code)

begin

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(undef, 6))

messages[1] = ruleSPGaussianMeanVarianceOutNPP(nothing, Message(Multivariate, PointMass, m=[1.0, 1.0]), Message(MatrixVariate, PointMass, m=[1.0 0.0; 0.0 1.0]))
messages[2] = ruleSPGaussianMeanVarianceMPNP(Message(Univariate, PointMass, m=data[:y]), nothing, Message(Univariate, PointMass, m=1))
messages[3] = ruleSPGaussianMeanVarianceOutNPP(nothing, Message(Multivariate, PointMass, m=[1.0, 1.0]), Message(MatrixVariate, PointMass, m=[2.0 0.0; 0.0 2.0]))
messages[4] = ruleSPBivariateSIn2MGN(messages[2], messages[3], messages[1], f, currentGraph().nodes[:bivariate_1].status, 1000)
messages[5] = ruleSPBivariateSIn1MNG(messages[2], messages[3], messages[1], f, currentGraph().nodes[:bivariate_1].status, 1000)
messages[6] = ruleSPBivariateSOutNGG(nothing, messages[3], messages[1], f, currentGraph().nodes[:bivariate_1].status, 1000)

marginals[:m] = messages[6].dist * messages[2].dist

In [39]:
# Define algorithm
eval(Meta.parse(source_code));

In [40]:
data = Dict(:y => 4.2)
marginals = step!(data)

Dict{Any,Any} with 3 entries:
  :m => SampleList(s=[-1.69, 0.81, 3.86, 3.75, 2.72, 5.43, 2.67, -0.15, -0.86, …
  :z => 𝒩(xi=[2.61, 2.61], w=[[1.46, 0.54][0.54, 1.46]])…
  :x => 𝒩(xi=[2.11, 2.11], w=[[0.92, 0.46][0.46, 0.92]])…